# 0501 초과, 미만, 이상, 이하, 같은 다름
- lt, gt, le, ge, eq, ne 메서드는 dataframe의 크기 비교를 수행하는 메서드
- less than, grater than, less equal, grater equal, equal, not equal

In [2]:
import pandas as pd
import numpy as np

In [3]:
col = ['col1', 'col2', 'col3']
row = ['A', 'B','C']
df =  pd.DataFrame(data=[[10,20,10],
                        [80,30,60],
                        [20,10,70]],index=row,columns=col)
print(df)

   col1  col2  col3
A    10    20    10
B    80    30    60
C    20    10    70


In [4]:
#스칼라값과의 비교
print(df.eq(10))

    col1   col2   col3
A   True  False   True
B  False  False  False
C  False   True  False


In [5]:
print(df.ne(20)) #20과 같지 않는 경우 true 표시

    col1   col2  col3
A   True  False  True
B   True   True  True
C  False   True  True


In [8]:
#series 와의 비교
s1 = pd.Series([10,30], index=["col1", "col3"])
print(s1)
print(df.gt(s1)) #col1에서 10이상, col3에서 30이상이면 true

col1    10
col3    30
dtype: int64
    col1   col2   col3
A  False  False  False
B   True  False   True
C   True  False   True


In [7]:
#만약 존재하지 않는 테이블 비교한다면, 레이블이 생성(broadcast)됨
s2 = pd.Series([10], index=["col4"])
print(df.lt(s2)) #df에는 col4가 없어 col4가 생성됨

    col1   col2   col3   col4
A  False  False  False  False
B  False  False  False  False
C  False  False  False  False


In [9]:
#axis에 따른 비교의 차이
print(df.le([10,20,30], axis="columns")) #열 기준으로 비교했을 때 각각 10,20,30 이하면 true

    col1   col2   col3
A   True   True   True
B  False  False  False
C  False   True  False


In [10]:
print(df.le([10,20,30], axis="index")) #행 기준으로 비교했을 때 각각 10,20,30이하면 true

    col1   col2   col3
A   True  False   True
B  False  False  False
C   True   True  False


In [11]:
#dataframe과의 비교
df2= pd.DataFrame([[50], [50], [50]], index=row, columns=['col1'])
print(df2)

   col1
A    50
B    50
C    50


In [12]:
print(df.ge(df2))

    col1   col2   col3
A  False  False  False
B   True  False  False
C  False  False  False


In [13]:
#멀티 인덱스의 사용(level인수)
row_mul = [['U','U','U','D','D','D'],['A','B','C','A','B','C']]
df_mul = pd.DataFrame(data=[[10,20,10],
                           [80,30,60],
                           [20,10,70],
                           [30,70,60],
                           [10,90,40],
                           [50,30,80]], index=row_mul, columns=col)
print(df_mul)

     col1  col2  col3
U A    10    20    10
  B    80    30    60
  C    20    10    70
D A    30    70    60
  B    10    90    40
  C    50    30    80


In [14]:
print(df.ge(df_mul, level=1))
#level=1이기 때문에, A,B,C를 index로 하는 두 dataframe과의 비교처럼 진행됨

      col1   col2   col3
U A   True   True   True
  B   True   True   True
  C   True   True   True
D A  False  False  False
  B   True  False   True
  C  False  False  False


# 0502 dtype 기반 열 선택

In [15]:
col1 = [1, 2, 3, 4, 5]
col2 = ['one', 'two', 'three', 'four', 'five']
col3 = [1.5, 2.5, 3.5, 4.5, 5.5]
col4 = [True, False, False, True, True]
df = pd.DataFrame({"col1": col1, "col2": col2, "col3": col3, "col4": col4})
print(df)

   col1   col2  col3   col4
0     1    one   1.5   True
1     2    two   2.5  False
2     3  three   3.5  False
3     4   four   4.5   True
4     5   five   5.5   True


In [16]:
print(df.dtypes)

col1      int64
col2     object
col3    float64
col4       bool
dtype: object


In [17]:
#include 사용
# include에 포함될 type을 입력 -> 해당 type인 열만 반환 가능
result = df.select_dtypes(include=[float, bool])
print(result)

   col3   col4
0   1.5   True
1   2.5  False
2   3.5  False
3   4.5   True
4   5.5   True


In [19]:
#exclude 사용 exclude에 제외할 type을 입력함으로써, 해당 type인 열만 제외하여 반환 가능
result = df.select_dtypes(exclude = ['int64'])
print(result)

    col2  col3   col4
0    one   1.5   True
1    two   2.5  False
2  three   3.5  False
3   four   4.5   True
4   five   5.5   True


In [20]:
#include &exclude 혼합 사용
result =df.select_dtypes(include=[float, object], exclude = ['int64'])
print(result)

    col2  col3
0    one   1.5
1    two   2.5
2  three   3.5
3   four   4.5
4   five   5.5


# 0503 임계값 적용

- clip 메서드는 series나 dataframe에 대해 요소범위를 제한함
- 상한선과 하한선은 임계값으로 정해서 임계값 밖의 값을 임계값으로 변경
- lower = 하한선
- upper = 상한선
- axis = 계산한 기준이 되는 레이블
- inplace = 제자리에서 계산할지 여부

In [21]:
col  = ['col1','col2','col3']
row  = ['row1','row2','row3']
data = [[-7,3,9],
        [6,-8,1],
        [-3,0,-7]]
df = pd.DataFrame(data,row,col)
print(df)

      col1  col2  col3
row1    -7     3     9
row2     6    -8     1
row3    -3     0    -7


In [22]:
#하한선 = -4, 상한선= 5
print(df.clip(-4,5))

      col1  col2  col3
row1    -4     3     5
row2     5    -4     1
row3    -3     0    -4


In [23]:
#임계값을 series로 설정
s = pd.Series(data=[1,2,3], index=row)
print(s)

row1    1
row2    2
row3    3
dtype: int64


In [24]:
print(df.clip(-s, s, axis=0))

      col1  col2  col3
row1    -1     1     1
row2     2    -2     1
row3    -3     0    -3


# 0504 레이블 필터링
- 레이블에 대해서 조건에 맞는 레이블만 필터링하는 메서드
- items : 이름으로 필터링
- like : str로 필터링함
- regex : 정규표현식으로 이용해 필터링

In [25]:
col  = ['alpha','beta','gamma','delta','epsilon']
row  = ['sigma','omega','lambda']
data = [[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15]]
df = pd.DataFrame(data,row,col)
print(df)

        alpha  beta  gamma  delta  epsilon
sigma       1     2      3      4        5
omega       6     7      8      9       10
lambda     11    12     13     14       15


In [26]:
#items인수를 사용하는 경우
print(df.filter(items = ['alpha', 'delta']))

        alpha  delta
sigma       1      4
omega       6      9
lambda     11     14


In [27]:
print(df.filter(items = ['omega'], axis=0)) #행 말하는 거

       alpha  beta  gamma  delta  epsilon
omega      6     7      8      9       10


In [28]:
#like인수 사용하는 경우
print(df.filter(like='ta'))

        beta  delta
sigma      2      4
omega      7      9
lambda    12     14


In [29]:
print(df.filter(regex='[mn]')) #m과 n이 포함된 것

        gamma  epsilon
sigma       3        5
omega       8       10
lambda     13       15


In [30]:
print(df.filter(regex = '^g')) #^ = ^뒤에 있는 문자로 시작하는 문자열

        gamma
sigma       3
omega       8
lambda     13


In [31]:
print(df.filter(regex ='a$')) #$ = $앞에 있는 문자로 끝내는 문자열

        alpha  beta  gamma  delta
sigma       1     2      3      4
omega       6     7      8      9
lambda     11    12     13     14


# 0505 샘플 추출

In [32]:
col  = ['col1','col2','col3']
row  = ['row1','row2','row3','row4','row5']
data = [[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15]]
df = pd.DataFrame(data,row,col)
print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7     8     9
row4    10    11    12
row5    13    14    15


In [33]:
#n을 설정해서 원하는 갯수의 랜덤 추출 가능
print(df.sample(2))

      col1  col2  col3
row4    10    11    12
row3     7     8     9


In [34]:
print(df.sample(10, replace=True)) #중복추출 사용 replace=True

      col1  col2  col3
row3     7     8     9
row4    10    11    12
row3     7     8     9
row3     7     8     9
row3     7     8     9
row2     4     5     6
row5    13    14    15
row2     4     5     6
row1     1     2     3
row1     1     2     3


In [35]:
#frac을 사용하는 경우, 전체에 대한 추출 비용 정함
print(df.sample(frac=0.4))

      col1  col2  col3
row2     4     5     6
row3     7     8     9


In [36]:
#weights를 통한 가중치의 사용
s=pd.Series(data=[10,10,3,3,1], index=row)
print(s)

row1    10
row2    10
row3     3
row4     3
row5     1
dtype: int64


In [37]:
print(df.sample(2, weights=s))

      col1  col2  col3
row1     1     2     3
row3     7     8     9


In [38]:
#random_state를 통한 동일값 재출력 허용
#random_state에 원하는 값을 설정하므로 출력 결과 동일하게 출력하는 것
print(df.sample(5, random_state=7))

      col1  col2  col3
row1     1     2     3
row4    10    11    12
row3     7     8     9
row2     4     5     6
row5    13    14    15


In [39]:
print(df.sample(5, random_state=7))

      col1  col2  col3
row1     1     2     3
row4    10    11    12
row3     7     8     9
row2     4     5     6
row5    13    14    15


In [40]:
#ignore_index의 사용
print(df.sample(3, ignore_index=True))

   col1  col2  col3
0     7     8     9
1     4     5     6
2    10    11    12
